### Intuition

General purpose of Boltzmann machines is recommendation systems.

General architecture:
- Visible and hidden nodes

- No output nodes

- Input nodes are all interconnected

- Boltzmann machines do not just expect input data, but they also generate data



In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [118]:
movies = pd.read_csv("ml-1m/movies.dat", sep = "::",
                    header = None, engine = "python",
                    encoding = "latin-1")
users = pd.read_csv("ml-1m/users.dat", sep = "::",
                    header = None, engine = "python",
                    encoding = "latin-1")
ratings = pd.read_csv("ml-1m/ratings.dat", sep = "::",
                    header = None, engine = "python",
                    encoding = "latin-1")
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [119]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [120]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [121]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [122]:
# We will have to make the training set into a matrix where 
# the rows are the user, columns are the movies, and the values inside are the ratings

# Getting the total number of users and movies
# Column 1 is the user, Column 2 is the movies
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

NameError: name 'testing_set_array' is not defined

In [ ]:
nb_users, nb_movies

In [ ]:
# Converting the data into an array with users in rows and movies in columns
# We will have to create a list with the sublist being every user with info. on their ratings for each movies

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
# Torches are only one type of datatype in a multi-dimensions
# Torches are a better way to create calculation 

training_set_array = torch.FloatTensor(training_set_array)
testing_set_array = torch.FloatTensor(testing_set_array)

In [ ]:
# The original dataset is binary (the user either liked the movie or did not)
# As of now, the tensor and arrays are rated from 1 through 5 
# Need to convert the ratings to binary (0 or 1)
# We have to convert the 0's in the array to -1, since the 0's meant that the user did not rate the movie
# However, the 0 will no longer mean that the 0 is non-existenting, rather, that the user did not like the movie
# The OR condition does not work for the torch

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [ ]:
# Creating a class for the Boltmann model
# We must initialize the biased and weights randomly
# One biases for each hidden node (we must create the 1 to make it into 2-dimensions)

class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


In [ ]:
# Using the model, trying to predict results for the testing set
# Training the RBM model

# Quick explanation: for every user in the testing set, we capture their visible node
# then, we check their rating for the movies they rated
# We use then sampled them for the hidden nodes
# And also, use those hidden nodes to sample the visible nodes


test_loss = 0
counter = 0.0
for id_user in range(nb_users): 
    v = training_set_array[id_user:id_user+1] # We need to use the training set to make the predictions
    vt = testing_set_array[id_user:id_user+1] # the target variable, original rating of the testing set
    if len(vt[vt>=0] > 0): # All the rating that are exisiting (not less than 0)
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)

        test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0])) # prediction is vk, only need the variables that are positive
        counter += 1.
print('loss: ' + str(test_loss/counter))